In [2]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter

In [1]:
from secret_case import API_TOKEN

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [10]:
url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":"AAPL,BTC-USD,EURUSD=X"}

headers = {
    'x-api-key': API_TOKEN
    }

response = rq.get(url, headers=headers, params=querystring).json()

print(response)

{'quoteResponse': {'result': [{'language': 'en-US', 'region': 'US', 'quoteType': 'EQUITY', 'typeDisp': 'Equity', 'quoteSourceName': 'Nasdaq Real Time Price', 'triggerable': True, 'customPriceAlertConfidence': 'HIGH', 'currency': 'USD', 'marketState': 'POSTPOST', 'regularMarketChangePercent': -0.88974994, 'regularMarketPrice': 170.43, 'epsTrailingTwelveMonths': 5.97, 'epsForward': 6.22, 'epsCurrentYear': 5.73, 'priceEpsCurrentYear': 29.743454, 'sharesOutstanding': 15634199552, 'bookValue': 3.852, 'fiftyDayAverage': 182.1924, 'fiftyDayAverageChange': -11.762405, 'fiftyDayAverageChangePercent': -0.064560354, 'twoHundredDayAverage': 165.9458, 'twoHundredDayAverageChange': 4.484192, 'twoHundredDayAverageChangePercent': 0.027022026, 'marketCap': 2664536473600, 'forwardPE': 27.400322, 'priceToBook': 44.244545, 'sourceInterval': 15, 'exchangeDataDelayedBy': 0, 'averageAnalystRating': '2.0 - Buy', 'tradeable': False, 'cryptoTradeable': False, 'postMarketChangePercent': -0.0234661, 'postMarketTi